In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

In [ ]:
# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')

In [ ]:
# Load GeoJSON file with Catalonian counties (comarcas)
map_catalonia='county_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)
geo_data

In [ ]:
# Translate column used to merge datasets
geo_data.rename(columns={'nomcomar': 'County_name'}, inplace=True)
# Sort array by column of interest
geo_data_sorted=geo_data.sort_values(by='County_name')

In [ ]:
# Sum up the participation for each county
participation_by_county = election_data.iloc[:, 11:21].groupby(by=election_data["County_name"]).sum()
participation_by_county.reset_index()
participation_by_county

In [ ]:
# Add a column to calculate the participation ratio (total number of voting-eligible population)
participation_by_county['Participation_rate'] = participation_by_county['Part_20:00'] / participation_by_county['Electoral_census']
participation_by_county

In [ ]:
# Rename Val d'Aran to correspond to name used in geographical data 
participation_by_county = participation_by_county.rename(index={"Consell General d'Aran": "Val d'Aran"})

In [ ]:
# Merge county map and participation by county
merged_data = geo_data_sorted.merge(participation_by_county, on='County_name')
merged_data

In [ ]:
named_colorscales = px.colors.named_colorscales()
print(named_colorscales)


In [ ]:
# Plot the participation rate on a map of Catalonian counties
fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Participation_rate',  # Change to the column you want to visualize
    hover_name='County_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    color_continuous_scale='viridis',
    zoom=7,
    title='Participation rate by county')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, width=800)
fig.show()